## Train

## Importing Libraries

In [1]:
import pkg_resources

REQUIRED_PACKAGES = [
    'mtcnn','tensorflow', 'keras-facenet', 'sklearn', 'scikit-learn'
]

for package in REQUIRED_PACKAGES:
    try:
        dist = pkg_resources.get_distribution(package)
        print('{} ({}) is already installed'.format(dist.key, dist.version))
    except pkg_resources.DistributionNotFound:
        !pip install {package}

mtcnn (0.1.1) is already installed
tensorflow (2.12.0) is already installed
keras-facenet (0.3.2) is already installed
sklearn (0.0.post5) is already installed
scikit-learn (1.2.2) is already installed


In [2]:
# Only for jupyter-notebook/Colab
!pip uninstall -y opencv-python
!pip install opencv-python-headless


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


## Extract Faces

In [3]:
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN

2023-06-04 12:09:03.801869: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)

    face_array = None
    if len(results) != 0:
        # extract the bounding box from the first face
        x1, y1, width, height = results[0]['box']
        # bug fix
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        # extract the face
        face = pixels[y1:y2, x1:x2]
        # resize pixels to the model size
        image = Image.fromarray(face)
        image = image.resize(required_size)
        face_array = asarray(image)
    
    return face_array

In [5]:
# load images and extract faces for all images in a directory
def load_faces(directory):
    faces = list()
    # enumerate files
    for filename in listdir(directory):
        # path
        path = directory + filename
        # get face
        face = extract_face(path)
        if face is not None:
            # store
            faces.append(face)
    return faces

In [6]:
# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
    X, y = list(), list()
    # enumerate folders, on per class
    for subdir in listdir(directory):
        # path
        path = directory + subdir + '/'
        # skip any files that might be in the dir
        if not isdir(path):
            continue
        # load all faces in the subdirectory
        faces = load_faces(path)
        # create labels
        labels = [subdir for _ in range(len(faces))]
        # summarize progress
        print('>loaded %d examples for class: %s' % (len(faces), subdir))
        # store
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)

In [7]:
# load train dataset
trainX, trainy = load_dataset('images/train/')
print(trainX.shape, trainy.shape)

# load test dataset
testX, testy = load_dataset('images/val/')

# save arrays to one file in compressed format
savez_compressed('ss.npz', trainX, trainy, testX, testy)

2023-06-04 12:09:08.280143: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-04 12:09:08.299697: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-04 12:09:08.299985: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-04 12:09:08.302033: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-04 12:09:08.302258: I tensorflow/compile

1/1 [==============================] - 4s 4s/step


2023-06-04 12:09:13.713565: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1/1 [==============================] - 0s 225ms/step
>loaded 3 examples for class: 1685_Anjana_Om_Kashyap
1/1 [==============================] - 0s 235ms/step
>loaded 3 examples for class: 1686_Barkha_Dutt
1/1 [==============================] - 0s 219ms/step
>loaded 3 examples for class: 1687_Ravish_Kumar
1/1 [==============================] - 0s 81ms/step
>loaded 4 examples for class: 1688_Arnab_Goswami
(13, 224, 224, 3) (13,)
1/1 [==============================] - 0s 273ms/step
>loaded 1 examples for class: 1685_Anjana_Om_Kashyap
1/1 [==============================] - 0s 81ms/step
>loaded 2 examples for class: 1686_Barkha_Dutt
1/1 [==============================] - 0s 89ms/step
>loaded 2 examples for class: 1687_Ravish_Kumar
1/1 [==============================] - 0s 81ms/step
>loaded 2 examples for class: 1688_Arnab_Goswami


## Embeddings

In [12]:
# calculate a face embedding for each face in the dataset using facenet
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
from keras_facenet import FaceNet
from tensorflow import keras
from sklearn.svm import SVC
from tensorflow import keras
import tensorflow as tf

In [13]:
embedder = FaceNet()

In [14]:
# get the face embedding for one face
def get_embedding(model, face_pixels):
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    detections = model.extract(face_pixels, threshold = 0.95)
    # standardize pixel values across channels (global)
    #mean, std = face_pixels.mean(), face_pixels.std()
    #face_pixels = (face_pixels - mean) / std
    # transform face into one sample
    samples = expand_dims(face_pixels, axis=0)
    # make prediction to get embedding
    # yhat = model.predict(samples)
    yhat = model.embeddings(samples)

    return yhat[0]

In [15]:
# load the face dataset
data = load('ss.npz')

trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)
# load the facenet model
# model = load_model('facenet_keras.h5')
# model.summary()
model = embedder
# model = SVC(kernel='linear', probability=True)
# model = load_model('face-rec_Google.npz')
# model = load('test.npz')
# model = tf.keras.applications.ResNet50(weights='imagenet')

print('Loaded Model')
# convert each face in the train set to an embedding

Loaded:  (13, 224, 224, 3) (13,) (7, 224, 224, 3) (7,)
Loaded Model


In [16]:
newTrainX = list()
for face_pixels in trainX:
    embedding = get_embedding(model, face_pixels)
    newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)
# convert each face in the test set to an embedding
newTestX = list()
for face_pixels in testX:
    embedding = get_embedding(model, face_pixels)
    newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)
# save arrays to one file in compressed format
savez_compressed('ss-embd.npz', newTrainX, trainy, newTestX, testy)

1/1 [==============================] - 0s 33ms/step
(13, 512)
1/1 [==============================] - 0s 27ms/step
(7, 512)


## KNN

In [17]:
# load the embedding dataset
data = load('ss-embd.npz')

trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']

In [18]:
# Create and train the KNN classifier
from sklearn import neighbors
import pickle

knn_clf = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance')
knn_clf.fit(trainX, trainy)

KNeighborsClassifier(weights='distance')

In [19]:
# Evaluate classifier with test set
y_pred = knn_clf.predict(testX)

knn_clf.score(testX,testy)

1.0

In [20]:
# Evaluate metrics
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(testy, y_pred))
print(confusion_matrix(testy, y_pred))

                        precision    recall  f1-score   support

1685_Anjana_Om_Kashyap       1.00      1.00      1.00         1
      1686_Barkha_Dutt       1.00      1.00      1.00         2
     1687_Ravish_Kumar       1.00      1.00      1.00         2
    1688_Arnab_Goswami       1.00      1.00      1.00         2

              accuracy                           1.00         7
             macro avg       1.00      1.00      1.00         7
          weighted avg       1.00      1.00      1.00         7

[[1 0 0 0]
 [0 2 0 0]
 [0 0 2 0]
 [0 0 0 2]]


In [21]:
# Save the trained KNN classifier
model_save_path = "trained_face_knn.clf"

with open(model_save_path, 'wb') as f:
    pickle.dump(knn_clf, f)

## Webcam

In [1]:
import cv2
import numpy as np
from mtcnn import MTCNN
from keras.models import load_model
from scipy.spatial.distance import cosine
import tensorflow as tf
from keras_facenet import FaceNet
from time import time
import pickle

2023-06-04 12:45:57.126079: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load the FaceNet model
facenet_model = FaceNet()

# Load KNN classifier
model_path = ("trained_face_knn.clf")
with open(model_path, 'rb') as f:
    knn_clf = pickle.load(f)

# Load the MTCNN face detection model
mtcnn_detector = MTCNN()

2023-06-04 12:46:01.678898: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-04 12:46:01.712493: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-04 12:46:01.713205: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-04 12:46:01.716959: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-04 12:46:01.717742: I tensorflow/compile

In [ ]:
# Open the video capture
video_capture = cv2.VideoCapture('raavish_kumar_1 - Trim.mp4')  # Use 0 for webcam or provide the path to a video file

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('raavish_kumar_1-Trim_knn_output.mp4', fourcc, video_capture.get(cv2.CAP_PROP_FPS), 
                      (int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)),  int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))))


while True:
    # Start timer
    start_time = time()
    
    # Read a frame from the video
    ret, frame = video_capture.read()

    if not ret:
        break

    # detection_time = time()
    # Detect faces in the frame using MTCNN
    faces = mtcnn_detector.detect_faces(frame)
    # print(f"detection-time: {time()-detection_time}")
    print(faces)

    # Iterate over detected faces
    for face in faces:
        # Extract the face coordinates
        x, y, w, h = face['box']

        # Extract the face ROI from the frame
        face_roi = frame[y:y+h, x:x+w]

        # Preprocess the face ROI
        face_roi = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
        face_roi = cv2.resize(face_roi, (224, 224))
        face_roi = face_roi.astype('float32')
        # mean, std = face_roi.mean(), face_roi.std()
        # face_roi = (face_roi - mean) / std

        # Convert the preprocessed face ROI to a 4D tensor
        face_tensor = np.expand_dims(face_roi, axis=0)

        # embedding_time = time()
        # Generate embeddings using the FaceNet model
        face_embedding = facenet_model.embeddings(face_tensor)[0]
        # print(f"embedding-time: {time()-embedding_time}")

        
        # Use the KNN model to predict the label for the face
        # prediction = knn_clf.predict(np.expand_dims(face_embedding, axis=0))
        # print(f"Prediction: {prediction}")

        # knn_predict_time = time()
        # Use the KNN model to predict the probability for each class
        proba = knn_clf.predict_proba(np.expand_dims(face_embedding, axis=0))
        print(f"Probability prediction: {proba}")
        # print(f"knn-predict-time: {time()-knn_predict_time}")

        # knn_kneighbors_time = time()
         # Use the KNN model to find distance to the best matches for the test face
        closest_distances = knn_clf.kneighbors(np.expand_dims(face_embedding, axis=0), n_neighbors=1)
        min_distance = closest_distances[0][0][0]
        print(f"Min Dsitance: {min_distance}")
        # print(f"knn_kneighbors_time: {time()-knn_kneighbors_time}")

        # Define a threshold for face recognition
        threshold = 0.75
        
        if min_distance < threshold:
            # Identify the recognized face
            recognized_name = knn_clf.classes_[proba.argmax()]
            print(f"Recognized Name: {recognized_name}")

            # Draw bounding box and label for the recognized face with confidence level
            label = f"{recognized_name} (Prob: {max(proba[0]):.2f})"
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        else:
 
            # Draw bounding box and label for unknown face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            cv2.putText(frame, 'Unknown', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    
    # end timer
    time_taken = time() - start_time
    fps = 1/time_taken
    cv2.putText(frame, f'Fps: {round(fps, 3)}', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    # print(f'FPS:{fps}')
    
    # Display the frame
    # cv2.imshow('Video', frame)
    out.write(frame)

    # Exit the loop if 'q' is pressed (commented to work with opencv-python-headless install)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

# Release the video capture and close all
video_capture.release()
out.release()

1/1 [==============================] - 0s 183ms/step
[]
1/1 [==============================] - 0s 302ms/step
[]
1/1 [==============================] - 0s 398ms/step
[]
1/1 [==============================] - 1s 840ms/step
[]
